In [ ]:
import cut_pic

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
# path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\短路\11812110280163_D4.jpg'  # 7320*3778
# path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\短路\11812110540646_C20,D20.jpg' # 6120*3778
# path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\明暗片\441402I1321688.jpg' # 7296*3696
# path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\明暗片\441420I1310954.jpg' # 7296*3754
# test = Image.open(path)

In [ ]:
test.size

**去除边界黑框**

cut_points = [60, 65, 7320-60, 3778-120]   # 针对size为7320*3778

cut_points = [65, 65, 6120-65, 3778-125]   # 针对size为6120*3778

cut_points = [50, 50, 7296-55, 3696-60]    # 针对size为7296*3696

cut_points = [50, 50, 7296-55, 3754-115]   # 针对size为7296*3754

In [ ]:
cut_points = [60, 65, 7320-60, 3778-120]
test = test.crop(cut_points)

In [ ]:
pic = np.asarray(test)

In [ ]:
pic.shape

In [ ]:
cut_pic.grid_cut(pic, 6, 24, True)

In [ ]:
cut_pic.get_cut_line(pic, 6, 24, True)

In [ ]:
plt.imshow(pic[0:1195, 0:600])

**灰度尺（20级）**

In [ ]:
gray_scale = np.linspace(1, 255, 20, endpoint=True).round()
gray_scale = np.asarray(gray_scale, dtype=int)
gray_scale

In [ ]:
gs=np.zeros((10,20,3))
gs.shape

In [ ]:
for i in range(len(gray_scale)):
    gs[:,i,:] = gray_scale[i] + gs[:,i,:]

In [ ]:
plt.imshow(gs)

**对切割下来的每一小块的像素值按照从大到小排列，求 wisorized mean，或者取其中位数代表该块硅片的灰度值**

In [ ]:
path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\明暗片\441402I1321688.jpg'
test = Image.open(path)
test.size

In [ ]:
def cut_edge(img):
    if img.size == (7296, 3696):
        img = img.crop([50, 50, 7296-55, 3696-60])
    else:
        img = img.crop([50, 50, 7296-55, 3754-115])
    return img

In [ ]:
test = cut_edge(test)

In [ ]:
test.size

In [ ]:
pic = np.asarray(test)

In [ ]:
pic.shape

In [ ]:
cut_points = cut_pic.get_cut_line(pic, 6, 12, True)

In [ ]:
cut_points

In [ ]:
cut_mat = np.zeros((6*12, 4), dtype=int)

In [ ]:
l = 0
for i in range(len(cut_points[1])-1):
    for j in range(len(cut_points[0])-1):
        tmp = [cut_points[1][i], cut_points[1][i+1], cut_points[0][j], cut_points[0][j+1]]
        cut_mat[l,:] = tmp
        l += 1

In [ ]:
cut_mat

In [ ]:
plt.imshow(pic[cut_mat[0,:][0]:cut_mat[0,:][1], cut_mat[0,:][2]:cut_mat[0,:][3], :])

In [ ]:
# tmp = pic[cut_mat[0,:][0]:cut_mat[0,:][1], cut_mat[0,:][2]:cut_mat[0,:][3], 0]

In [ ]:
med = np.zeros((1, 6*12), dtype=int)
for i in range(6*12):
    tmp = pic[cut_mat[i,:][0]:cut_mat[i,:][1], cut_mat[i,:][2]:cut_mat[i,:][3], 0]
    med[0,i] = np.median(tmp.ravel())    

In [ ]:
# tmp = np.abs(118 - gray_scale)

In [ ]:
# np.where(tmp == np.min(tmp))[0][0]

In [ ]:
def match_gray_scale(pixel):
    tmp = np.abs(pixel - gray_scale)
    return np.where(tmp == np.min(tmp))[0][0]

In [ ]:
results = []
for i in range(72):
    results.append(match_gray_scale(med[0][i]))

In [ ]:
np.max(results)-np.min(results)

In [ ]:
from scipy import stats

In [ ]:
stats.mode(results)[0][0]

In [ ]:
from collections import Counter

In [ ]:
results.ravel()

In [ ]:
freq = Counter(results.ravel())
freq

In [ ]:
freq.most_common()[-1][0]

In [ ]:
results = np.array(results).reshape((1, 6*12))

In [ ]:
np.where(results == 8)

In [ ]:
loc = np.where(results == 9)[1]
loc

In [ ]:
loc[1]

In [ ]:
len(loc)

In [ ]:
for i in range(len(loc)):
    plt.figure(i)
    plt.imshow(pic[cut_mat[loc[i],:][0]:cut_mat[loc[i],:][1], cut_mat[loc[i],:][2]:cut_mat[loc[i],:][3], :])

In [ ]:
cut_mat[33,:]

In [ ]:
plt.imshow(pic[cut_mat[7,:][0]:cut_mat[7,:][1], cut_mat[7,:][2]:cut_mat[7,:][3], :])

## 封装成函数

In [1]:
import cut_pic
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy import stats
from collections import Counter

In [9]:
import os

In [12]:
pic_path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\明暗片'
pic_names = os.listdir(pic_path)

In [21]:
paths = [pic_path + '\\'+name for name in pic_names]

In [2]:
# path =r'C:\Users\leyuan\Desktop\second term of second postgraduate year\数据诊所\光伏组件短路检测\明暗片\441402I1321688.jpg'

In [7]:
## 灰度尺
gray_scale = np.linspace(1, 255, 20, endpoint=True).round()
gray_scale = np.asarray(gray_scale, dtype=int)

In [3]:
def cut_edge(img):
    if img.size == (7296, 3696):
        img = img.crop([50, 50, 7296-55, 3696-60])
    else:
        img = img.crop([50, 50, 7296-55, 3754-115])
    return img

In [4]:
def match_gray_scale(pixel):
    tmp = np.abs(pixel - gray_scale)
    return np.where(tmp == np.min(tmp))[0][0]

In [37]:
def light_or_dark(path, rows, cols):
    img = Image.open(path)
    img = cut_edge(img)
    pic = np.asarray(img)
    cut_points = cut_pic.get_cut_line(pic, rows, cols, True)
    cut_mat = np.zeros((rows*cols, 4), dtype=int)
    l = 0
    for i in range(len(cut_points[1])-1):
        for j in range(len(cut_points[0])-1):
            tmp = [cut_points[1][i], cut_points[1][i+1], cut_points[0][j], cut_points[0][j+1]]
            cut_mat[l,:] = tmp
            l += 1
    med = np.zeros((1, rows*cols), dtype=int)
    for i in range(rows*cols):
        tmp = pic[cut_mat[i,:][0]:cut_mat[i,:][1], cut_mat[i,:][2]:cut_mat[i,:][3], 0]
        med[0,i] = np.median(tmp.ravel())
    results = []
    for i in range(rows*cols):
        results.append(match_gray_scale(med[0][i]))
    results = np.array(results).reshape((1, rows*cols))
    gray_range = np.max(results)-np.min(results)
    freq = Counter(results.ravel())
    index = freq.most_common()[-1][0]
    loc = np.where(results == index)[1]
    #for i in range(len(loc)):
        #plt.imshow(pic[cut_mat[loc[i],:][0]:cut_mat[loc[i],:][1], cut_mat[loc[i],:][2]:cut_mat[loc[i],:][3], :])
    if gray_range >= 5:
        qualified = False
    else:
        qualified = True
    return list([qualified, cut_mat, freq, loc])
    #return qualified

In [38]:
light_or_dark(paths[9], 6, 12)

[True, array([[   0,  598,    0,  597],
        [   0,  598,  597, 1197],
        [   0,  598, 1197, 1798],
        [   0,  598, 1798, 2397],
        [   0,  598, 2397, 2996],
        [   0,  598, 2996, 3598],
        [   0,  598, 3598, 4196],
        [   0,  598, 4196, 4798],
        [   0,  598, 4798, 5398],
        [   0,  598, 5398, 5997],
        [   0,  598, 5997, 6597],
        [   0,  598, 6597, 7190],
        [ 598, 1197,    0,  597],
        [ 598, 1197,  597, 1197],
        [ 598, 1197, 1197, 1798],
        [ 598, 1197, 1798, 2397],
        [ 598, 1197, 2397, 2996],
        [ 598, 1197, 2996, 3598],
        [ 598, 1197, 3598, 4196],
        [ 598, 1197, 4196, 4798],
        [ 598, 1197, 4798, 5398],
        [ 598, 1197, 5398, 5997],
        [ 598, 1197, 5997, 6597],
        [ 598, 1197, 6597, 7190],
        [1197, 1799,    0,  597],
        [1197, 1799,  597, 1197],
        [1197, 1799, 1197, 1798],
        [1197, 1799, 1798, 2397],
        [1197, 1799, 2397, 2996],
        

In [32]:
for path in paths:
    print(light_or_dark(path, 6, 12))

False
False
False
False
False
False
False
False
False
True
False
False
False
False
